In [1]:
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from sharadar.pipeline.factors import (
    MarketCap,
    EV,
    Fundamentals,
    FundamentalsSeries
)
from sharadar.pipeline.engine import load_sharadar_bundle, symbol, symbols, make_pipeline_engine
from zipline.pipeline.filters import StaticAssets
import time
import datetime
import math
from zipline.utils.calendars import get_calendar
from pandas.tseries.offsets import DateOffset
import numpy as np
from zipline.pipeline.factors import CustomFactor, Returns
from sharadar.pipeline.engine import WithAssetFinder

pd.options.display.float_format = '{:.0f}'.format

In [2]:
bundle = load_sharadar_bundle()

In [3]:
pipe = Pipeline(columns={
    'revenue1': Fundamentals(field='revenue', window_length=1),
    'revenue2': Fundamentals(field='revenue', window_length=2),
    'revenue3': Fundamentals(field='revenue', window_length=3),
    'revenue4': Fundamentals(field='revenue', window_length=4)
},
screen = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
)

engine = make_pipeline_engine()
pipe_start = pd.to_datetime('2020-06-01', utc=True)
pipe_end = pd.to_datetime('2020-06-01', utc=True)
stocks = engine.run_pipeline(pipe, pipe_start, pipe_end)
stocks

#AAPL 58,313 91,819 64,040 53,809
#IBM  17,571 21,776 18,028 19,161
#F    34,320 39,715 36,990 38,853



[2020-06-08 13:45:02.393338] INFO: sharadar_db_bundle: Compute pipeline values in chunks of 120 days.
[2020-06-08 13:45:02.395245] INFO: sharadar_db_bundle: Compute values for pipeline from 2020-06-01 to 2020-06-01.
[2020-06-08 13:45:02.907222] INFO: sharadar_db_bundle: Computing term 1 of 6 [AssetExists()]
[2020-06-08 13:45:02.908176] INFO: sharadar_db_bundle: Term already in workspace: no computation needed
[2020-06-08 13:45:02.909464] INFO: sharadar_db_bundle: Computing term 2 of 6 [Fundamentals{'field': 'revenue'}]
[2020-06-08 13:45:06.478762] INFO: sharadar_db_bundle: Elapsed time: 0:00:03.569269
[2020-06-08 13:45:06.479535] INFO: sharadar_db_bundle: Computing term 3 of 6 [Fundamentals{'field': 'revenue'}]
[2020-06-08 13:45:07.643127] INFO: sharadar_db_bundle: Elapsed time: 0:00:01.163565
[2020-06-08 13:45:07.644225] INFO: sharadar_db_bundle: Computing term 4 of 6 [Fundamentals{'field': 'revenue'}]
[2020-06-08 13:45:09.865728] INFO: sharadar_db_bundle: Elapsed time: 0:00:02.221475

revenue1    revenue2  \
2020-06-01 00:00:00+00:00 Equity(199059 [AAPL]) 58313000000 91819000000   
                          Equity(199623 [IBM])  17571000000 21777000000   
                          Equity(199713 [F])    34320000000 39715000000   

                                                   revenue3    revenue4  
2020-06-01 00:00:00+00:00 Equity(199059 [AAPL]) 64040000000 53809000000  
                          Equity(199623 [IBM])  18028000000 19161000000  
                          Equity(199713 [F])    36990000000 38853000000

In [4]:
assets = [199623, 199059, 199713]
today = pd.to_datetime('2020-06-01', utc=True)
window_length = 4

In [5]:
def get_fundamentals_df_window_length(self, sids, field_name, as_of_date=None, window_length=1):
    if as_of_date is None:
        as_of_date = pd.Timestamp.today()
    start_date = as_of_date - DateOffset(months=(window_length+1)*3)
    sql = "SELECT * FROM (SELECT ROW_NUMBER() OVER (PARTITION BY sid ORDER BY start_date DESC) row_num," \
    "sid, start_date, value FROM equity_supplementary_mappings WHERE sid IN (%s) AND field = '%s'" \
    "AND start_date >= %d AND start_date <= %d) t WHERE row_num <= %d"
    cmd = sql % (', '.join(map(str, sids)), field_name, start_date.value, as_of_date.value, window_length)
    print(cmd)     
        
    df = pd.read_sql_query(cmd, self.engine)
    
    df = df.pivot(index='row_num', columns='sid', values='value')
    
    df = df.reindex(columns=sids)
    
    return df.values.astype('float64')

a = get_fundamentals_df_window_length(bundle.asset_finder, assets, 'revenue_arq', today, window_length)
a
#             t      t-1    t-2    t-3  
#AAPL(199059) 58,313 91,819 64,040 53,809
#IBM(199623)  17,571 21,776 18,028 19,161
#F(199713)    34,320 39,715 36,990 38,853

# IBM(199623)   AAPL(199059)   F(199713)
#t     17,571         58,313      34,320
#t-1   21,776         91,819      39,715
#t-2   18,028         64,040      36,990
#t-3   19,161         53,809      38,853


SELECT * FROM (SELECT ROW_NUMBER() OVER (PARTITION BY sid ORDER BY start_date DESC) row_num,sid, start_date, value FROM equity_supplementary_mappings WHERE sid IN (199623, 199059, 199713) AND field = 'revenue_arq'AND start_date >= 1551398400000000000 AND start_date <= 1590969600000000000) t WHERE row_num <= 4


array([[1.7571e+10, 5.8313e+10, 3.4320e+10],
       [2.1777e+10, 9.1819e+10, 3.9715e+10],
       [1.8028e+10, 6.4040e+10, 3.6990e+10],
       [1.9161e+10, 5.3809e+10, 3.8853e+10]])

In [26]:
def get_fundamentals_df_window_length_test(self, sids, field_name, as_of_date=None, window_length=1):
    if as_of_date is None:
        as_of_date = pd.Timestamp.today()
    start_date = as_of_date - DateOffset(months=(window_length+1)*3)
    sql = "SELECT * FROM (SELECT ROW_NUMBER() OVER (PARTITION BY sid ORDER BY start_date DESC) row_num," \
    "sid, start_date, value FROM equity_supplementary_mappings WHERE sid IN (%s) AND field = '%s'" \
    "AND start_date >= %d AND start_date <= %d) t WHERE row_num <= %d"
    cmd = sql % (', '.join(map(str, sids)), field_name, start_date.value, as_of_date.value, window_length)
    print(cmd)     
        
    df = pd.read_sql_query(cmd, self.engine)
    df['date'] = pd.to_datetime(df['start_date'], utc=True)
    
    return df

get_fundamentals_df_window_length_test(bundle.asset_finder, assets, 'ps_arq', pd.to_datetime('2000-06-01', utc=True), window_length)


SELECT * FROM (SELECT ROW_NUMBER() OVER (PARTITION BY sid ORDER BY start_date DESC) row_num,sid, start_date, value FROM equity_supplementary_mappings WHERE sid IN (199623, 199059, 199713) AND field = 'ps_arq'AND start_date >= 794016000000000000 AND start_date <= 959817600000000000) t WHERE row_num <= 20


,row_num,sid,start_date,value,date
0,1,199059,958089600000000000,2.33,2000-05-12 00:00:00+00:00
1,2,199059,949449600000000000,2.395,2000-02-02 00:00:00+00:00
2,3,199059,945907200000000000,2.6260000000000003,1999-12-23 00:00:00+00:00
3,4,199059,933984000000000000,1.37,1999-08-07 00:00:00+00:00
4,5,199059,926467200000000000,0.9890000000000001,1999-05-12 00:00:00+00:00
5,6,199059,918518400000000000,0.848,1999-02-09 00:00:00+00:00
6,7,199059,914457600000000000,0.907,1998-12-24 00:00:00+00:00
7,8,199059,881366400000000000,0.28600000000000003,1997-12-06 00:00:00+00:00
8,1,199623,958176000000000000,2.1390000000000002,2000-05-13 00:00:00+00:00
9,2,199623,952992000000000000,2.206,2000-03-14 00:00:00+00:00


In [6]:
get_fundamentals_df_window_length(bundle.asset_finder, assets, 'revenue_arq', today, 20)

SELECT * FROM (SELECT ROW_NUMBER() OVER (PARTITION BY sid ORDER BY start_date DESC) row_num,sid, start_date, value FROM equity_supplementary_mappings WHERE sid IN (199623, 199059, 199713) AND field = 'revenue_arq'AND start_date >= 1425168000000000000 AND start_date <= 1590969600000000000) t WHERE row_num <= 20


array([[1.7571e+10, 5.8313e+10, 3.4320e+10],
       [2.1777e+10, 9.1819e+10, 3.9715e+10],
       [1.8028e+10, 6.4040e+10, 3.6990e+10],
       [1.9161e+10, 5.3809e+10, 3.8853e+10],
       [1.8182e+10, 5.8015e+10, 4.0342e+10],
       [2.1761e+10, 8.4310e+10, 4.1793e+10],
       [1.8756e+10, 6.2900e+10, 3.7666e+10],
       [2.0003e+10, 5.3265e+10, 3.8920e+10],
       [1.9072e+10, 6.1137e+10, 4.1959e+10],
       [2.2542e+10, 8.8293e+10, 4.1326e+10],
       [1.9153e+10, 5.2579e+10, 3.6451e+10],
       [1.9289e+10, 4.5408e+10, 3.9853e+10],
       [1.8155e+10, 5.2896e+10, 3.9146e+10],
       [2.1770e+10, 7.8351e+10, 3.8654e+10],
       [1.9226e+10, 4.6852e+10, 3.5943e+10],
       [2.0238e+10, 4.2358e+10, 3.9485e+10],
       [1.8684e+10, 5.0557e+10, 3.7718e+10],
       [2.2059e+10, 7.5872e+10, 4.0251e+10],
       [1.9280e+10, 5.1501e+10, 3.8144e+10],
       [2.0813e+10, 4.9605e+10, 3.7263e+10]])

In [7]:
symbol('DELL').to_dict()

{'sid': 124114,
 'symbol': 'DELL',
 'asset_name': 'Dell Technologies Inc',
 'start_date': Timestamp('2016-09-07 00:00:00+0000', tz='UTC'),
 'end_date': Timestamp('2020-06-02 00:00:00+0000', tz='UTC'),
 'first_traded': Timestamp('2016-09-07 00:00:00+0000', tz='UTC'),
 'auto_close_date': Timestamp('2020-06-03 00:00:00+0000', tz='UTC'),
 'exchange': 'The New York Stock Exchange',
 'exchange_full': 'NYSE',
 'tick_size': 0.01,
 'multiplier': 1.0,
 'exchange_info': ExchangeInfo('NYSE', 'The New York Stock Exchange', 'US')}

In [8]:
assets = [199623, 124114, 199713]
start = pd.to_datetime('2017-09-07', utc=True)
window_length = 20
get_fundamentals_df_window_length(bundle.asset_finder, assets, 'revenue_arq', start, 20)

SELECT * FROM (SELECT ROW_NUMBER() OVER (PARTITION BY sid ORDER BY start_date DESC) row_num,sid, start_date, value FROM equity_supplementary_mappings WHERE sid IN (199623, 124114, 199713) AND field = 'revenue_arq'AND start_date >= 1339027200000000000 AND start_date <= 1504742400000000000) t WHERE row_num <= 20


array([[1.9289e+10, 1.7816e+10, 3.9853e+10],
       [1.8155e+10, 2.0074e+10, 3.9146e+10],
       [2.1770e+10, 1.6247e+10, 3.8654e+10],
       [1.9226e+10, 1.3050e+10, 3.5943e+10],
       [2.0238e+10, 1.2534e+10, 3.9485e+10],
       [1.8684e+10,        nan, 3.7718e+10],
       [2.2059e+10,        nan, 4.0251e+10],
       [1.9280e+10,        nan, 3.8144e+10],
       [2.0813e+10,        nan, 3.7263e+10],
       [1.9590e+10,        nan, 3.3900e+10],
       [2.4113e+10,        nan, 3.5870e+10],
       [2.2397e+10,        nan, 3.4920e+10],
       [2.4364e+10,        nan, 3.7411e+10],
       [2.2484e+10,        nan, 3.5876e+10],
       [2.7699e+10,        nan, 3.7036e+10],
       [2.3720e+10,        nan, 3.5976e+10],
       [2.4924e+10,        nan, 3.8095e+10],
       [2.3408e+10,        nan, 3.5810e+10],
       [2.9304e+10,        nan, 3.6424e+10],
       [2.4747e+10,        nan, 3.2172e+10]])

In [9]:
symbol('LLEN').to_dict()

{'sid': 195848,
 'symbol': 'LLEN',
 'asset_name': 'L & L Energy Inc',
 'start_date': Timestamp('2008-08-04 00:00:00+0000', tz='UTC'),
 'end_date': Timestamp('2015-07-02 00:00:00+0000', tz='UTC'),
 'first_traded': Timestamp('2008-08-04 00:00:00+0000', tz='UTC'),
 'auto_close_date': Timestamp('2015-07-03 00:00:00+0000', tz='UTC'),
 'exchange': 'National Association of Securities Dealers Automated Quotation',
 'exchange_full': 'NASDAQ',
 'tick_size': 0.01,
 'multiplier': 1.0,
 'exchange_info': ExchangeInfo('NASDAQ', 'National Association of Securities Dealers Automated Quotation', 'US')}

In [10]:
# IBM, AAPL, F
assets = [199623, 199059, 199713]
start = pd.to_datetime('2017-09-07', utc=True)
window_length = 20
Y = get_fundamentals_df_window_length(bundle.asset_finder, assets, 'revenue_arq', start, 20)
Y

SELECT * FROM (SELECT ROW_NUMBER() OVER (PARTITION BY sid ORDER BY start_date DESC) row_num,sid, start_date, value FROM equity_supplementary_mappings WHERE sid IN (199623, 199059, 199713) AND field = 'revenue_arq'AND start_date >= 1339027200000000000 AND start_date <= 1504742400000000000) t WHERE row_num <= 20


array([[1.9289e+10, 4.5408e+10, 3.9853e+10],
       [1.8155e+10, 5.2896e+10, 3.9146e+10],
       [2.1770e+10, 7.8351e+10, 3.8654e+10],
       [1.9226e+10, 4.6852e+10, 3.5943e+10],
       [2.0238e+10, 4.2358e+10, 3.9485e+10],
       [1.8684e+10, 5.0557e+10, 3.7718e+10],
       [2.2059e+10, 7.5872e+10, 4.0251e+10],
       [1.9280e+10, 5.1501e+10, 3.8144e+10],
       [2.0813e+10, 4.9605e+10, 3.7263e+10],
       [1.9590e+10, 5.8010e+10, 3.3900e+10],
       [2.4113e+10, 7.4599e+10, 3.5870e+10],
       [2.2397e+10, 4.2123e+10, 3.4920e+10],
       [2.4364e+10, 3.7432e+10, 3.7411e+10],
       [2.2484e+10, 4.5646e+10, 3.5876e+10],
       [2.7699e+10, 5.7594e+10, 3.7036e+10],
       [2.3720e+10, 3.7472e+10, 3.5976e+10],
       [2.4924e+10, 3.5323e+10, 3.8095e+10],
       [2.3408e+10, 4.3603e+10, 3.5810e+10],
       [2.9304e+10, 5.4512e+10, 3.6424e+10],
       [2.4747e+10, 3.5966e+10, 3.2172e+10]])

In [11]:
def time_trend(Y, allowed_missing=0):
    """
    If 'allowed_missing' is zero, interpolate to fill the NaN.
    If all values are NaN, replace them with zero
    """
    if allowed_missing == 0:
        # interpolate is too slow for the Algo Platform
        # Y = pd.DataFrame(Y).interpolate().fillna(method='bfill').fillna(0)
        Y = pd.DataFrame(Y).fillna(method='ffill', axis=0).fillna(0)
    n = Y.shape[0]
    m = Y.shape[1]
    
    print("n=%d" % n)
    
    # idx: n-1 to 0
    idx = np.arange(start=(n - 1), stop=-1, step=-1, dtype=float)
    X = np.full((m, n), idx).T
    
    # shape: (N, M)
    X = np.where(np.isnan(Y), np.nan, X)

    X_mean = np.nanmean(X, axis=0)
    Y_mean = np.nanmean(Y, axis=0)

    # shape: (M,)
    XY_cov = np.nanmean((X - X_mean) * (Y - Y_mean), axis=0)

    X_var = np.nanvar(X, axis=0)

    # shape: (M,)
    beta = np.divide(XY_cov, X_var)
    alpha = Y_mean - beta * X_mean
    Y_est = alpha + np.multiply(beta, X)
    residual = Y - Y_est
    s2 = np.nansum(residual ** 2, axis=0) / (n - 2.0)
    std_err2 = s2 / (n * X_var)
    std_err = np.sqrt(std_err2)

    # Write nans back to locations where we have more
    # then allowed number of missing entries.
    nanlocs = np.isnan(X).sum(axis=0) > allowed_missing
    beta[nanlocs] = np.nan
    # alpha[nanlocs] = np.nan
    std_err[nanlocs] = np.nan

    # return (alpha, beta)
    return (beta, std_err)

beta, std_err = time_trend(Y)
beta

n=20


array([-4.12771429e+08,  8.27984962e+08,  2.18784211e+08])

In [12]:
n = 20
m = 3
X = np.arange(start=(n - 1), stop=-1, step=-1, dtype=float)
X

array([19., 18., 17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,
        6.,  5.,  4.,  3.,  2.,  1.,  0.])

In [13]:
import statsmodels.api as sm
import numpy as np

X = sm.add_constant(X)
model = sm.OLS(Y[:,1],X)
results = model.fit()
results.params

array([4.29181429e+10, 8.27984962e+08])

In [14]:
def logscale(x):
    # Given: y=log(1+x), y≈x when x is small (less than 1).
    return np.sign(x) * np.nan_to_num(np.log(np.abs(x + np.sign(x))))

        
class TimeTrend(CustomFactor, WithAssetFinder):
    inputs = []
    outputs = ['trend', 'std_err']
    # 20 quarters = 5 years
    window_length = 20
    params = ('field',)

    def compute(self, today, assets, out, field):
        field_name = field
        if '_' not in field_name:
            field_name += '_arq'
        y = self.asset_finder().get_fundamentals_df_window_length(assets, field_name, today, self.window_length)
        
        (out.trend, out.std_err) = time_trend(y)
        

class LogTimeTrend(TimeTrend):

    def compute(self, today, assets, out, field):
        y = logscale(data)

        (out.trend, out.std_err) = time_trend(y)

        # The arctan of a slope is the the angle θ with the origin between −π/2 and π/2
        # Then divide by π/2 to get a measure in [-1,1]
        out.trend = 2.0 * np.arctan(out.trend) / np.pi

In [15]:
universe = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
pipe = Pipeline(columns={
    'revenue': Fundamentals(field='revenue', window_length=1),
    'sales_trend': TimeTrend(field='revenue', mask=universe).trend,
},
screen = universe
)

engine = make_pipeline_engine()
stocks = engine.run_pipeline(pipe, start, start)
stocks

[2020-06-08 13:45:11.729867] INFO: sharadar_db_bundle: Compute pipeline values in chunks of 120 days.
[2020-06-08 13:45:11.731017] INFO: sharadar_db_bundle: Compute values for pipeline from 2017-09-07 to 2017-09-07.
[2020-06-08 13:45:11.736853] INFO: sharadar_db_bundle: Computing term 1 of 5 [AssetExists()]
[2020-06-08 13:45:11.737699] INFO: sharadar_db_bundle: Term already in workspace: no computation needed
[2020-06-08 13:45:11.738508] INFO: sharadar_db_bundle: Computing term 2 of 5 [Fundamentals{'field': 'revenue'}]
[2020-06-08 13:45:13.345389] INFO: sharadar_db_bundle: Elapsed time: 0:00:01.606869
[2020-06-08 13:45:13.346523] INFO: sharadar_db_bundle: Computing term 3 of 5 [StaticAssets([], 0)]
[2020-06-08 13:45:13.348361] INFO: sharadar_db_bundle: Elapsed time: 0:00:00.001858
[2020-06-08 13:45:13.349550] INFO: sharadar_db_bundle: Computing term 4 of 5 [TimeTrend([], 20)]
n=20
[2020-06-08 13:45:13.360590] INFO: sharadar_db_bundle: Elapsed time: 0:00:00.011051
[2020-06-08 13:45:13.3

revenue  sales_trend
2017-09-07 00:00:00+00:00 Equity(199059 [AAPL]) 45408000000    827984962
                          Equity(199623 [IBM])  19289000000   -412771429
                          Equity(199713 [F])    39853000000    218784211

In [38]:
class TimeTrend(CustomFactor):
    outputs = ['trend', 'std_err']
    window_length = 260
   
 
    def compute(self, today, assets, out, data):
        (out.trend, out.std_err) = time_trend(data)
        
class EarningYield(CustomFactor):
    inputs = [
        Fundamentals(field='netinccmnusd_art'),
        MarketCap()
    ]
    window_safe = True
    window_length = 1

    def compute(self, today, assets, out, earnings, mkt_cap):
        l = self.window_length
        out[:] = earnings[-l] / mkt_cap[-l]
        
universe = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
pipe = Pipeline(columns={
    'ey': 100*EarningYield(),
    'ev_trend': TimeTrend([EarningYield()]),
    'earnings': Fundamentals(field='netinccmnusd_art'),
    'ev': EV(),
    'mkt_cap': MarketCap()
},
screen = universe
)

engine = make_pipeline_engine()

pipe_start = pd.to_datetime('2020-05-04', utc=True)
pipe_end = pd.to_datetime('2020-05-06', utc=True)
stocks = engine.run_pipeline(pipe, pipe_start, pipe_end)
stocks

[2020-06-08 21:41:44.652535] INFO: sharadar_db_bundle: Compute pipeline values in chunks of 120 days.
[2020-06-08 21:41:44.653493] INFO: sharadar_db_bundle: Compute values for pipeline from 2020-05-04 to 2020-05-06.
[2020-06-08 21:41:44.693643] INFO: sharadar_db_bundle: Computing term 1 of 13 [AssetExists()]
[2020-06-08 21:41:44.694723] INFO: sharadar_db_bundle: Term already in workspace: no computation needed
[2020-06-08 21:41:44.695568] INFO: sharadar_db_bundle: Computing term 2 of 13 [Fundamentals{'field': 'netinccmnusd_art'}]
[2020-06-08 21:41:44.831701] INFO: sharadar_db_bundle: Elapsed time: 0:00:00.136142
[2020-06-08 21:41:44.832420] INFO: sharadar_db_bundle: Computing term 3 of 13 [Fundamentals{'field': 'cashnequsd_arq'}]
[2020-06-08 21:41:44.835009] INFO: sharadar_db_bundle: Elapsed time: 0:00:00.002594
[2020-06-08 21:41:44.835562] INFO: sharadar_db_bundle: Computing term 4 of 13 [StaticAssets([], 0)]
[2020-06-08 21:41:44.837054] INFO: sharadar_db_bundle: Elapsed time: 0:00:00

earnings            ev  \
2020-05-04 00:00:00+00:00 Equity(199059 [AAPL]) 57215000000 1322259218450   
                          Equity(199623 [IBM])   9015000000  166290392800   
                          Equity(199713 [F])    -3092000000  160922036520   
2020-05-05 00:00:00+00:00 Equity(199059 [AAPL]) 57215000000 1339986648600   
                          Equity(199623 [IBM])   9015000000  166121693328   
                          Equity(199713 [F])    -3092000000  160683414123   
2020-05-06 00:00:00+00:00 Equity(199059 [AAPL]) 57215000000 1359057722600   
                          Equity(199623 [IBM])   9015000000  166920796089   
                          Equity(199713 [F])    -3092000000  161120888517   

                                                                                         ev_trend  \
2020-05-04 00:00:00+00:00 Equity(199059 [AAPL])    (9.263304990577415e-05, 3.992761110214798e-06)   
                          Equity(199623 [IBM])    (-4.795119687116291e-05, 8.295634936025119e-06)   
                          Equity(199713 [F])     (0.00038173283042671537, 1.5853861166677376e-05)   
2020-05-05 00:00:00+00:00 Equity(199059 [AAPL])     (9.266280561297778e-05, 3.99183124546465e-06)   
                          Equity(199623 [IBM])     (-4.87677999413465e-05, 8.297145805274611e-06)   
                          Equity(199713 [F])       (0.0003951691609210266, 1.771651742626897e-05)   
2020-05-06 00:00:00+00:00 Equity(199059 [AAPL])    (9.273233830148895e-05, 3.989851518421369e-06)   
                          Equity(199623 [IBM])    (-4.954273400909246e-05, 8.297097909206391e-06)   
                          Equity(199713 [F])     (0.00040818020299151857, 1.9272857689127983e-05)   

                                                 ey       mkt_cap  
2020-05-04 00:00:00+00:00 Equity(199059 [AAPL])   5 1252926218450  
                          Equity(199623 [IBM])    8  108207392800  
                          Equity(199713 [F])    -16   19567036520  
2020-05-05 00:00:00+00:00 Equity(199059 [AAPL])   5 1270653648600  
                          Equity(199623 [IBM])    8  108038693328  
                          Equity(199713 [F])    -16   19328414123  
2020-05-06 00:00:00+00:00 Equity(199059 [AAPL])   4 1289724722600  
                          Equity(199623 [IBM])    8  108837796089  
                          Equity(199713 [F])    -16   19765888517